<a href="https://colab.research.google.com/github/fenikowski/03MAIR--Algoritmos-de-Optimizacion--2020/blob/master/Igor_Fenikowski_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Nombre: Igor Fenikowski
# https://colab.research.google.com/drive/1Y2OtF4drywGZMKtBOICsgVb3O3lq1D_h?usp=sharing
# https://github.com/fenikowski/03MAIR--Algoritmos-de-Optimizacion--2020/blob/master/Igor_Fenikowski_AG3.ipynb

In [ ]:
!pip install request
!pip install tsplib95

In [3]:
import urllib
import tsplib95
import random
from math import e, exp

In [4]:
file = "swiss42.tsp"

urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file)

('swiss42.tsp', <http.client.HTTPMessage at 0x7f184a21e198>)

In [5]:
problem = tsplib95.load(file)

nodes = list(problem.get_nodes())
edges = list(problem.get_edges())

### Busqueda aleatoria

In [6]:
def create_solution(nodes):
  solution = [nodes[0]]
  for i in nodes[1:]:
    solution = solution + [random.choice(list(set(nodes) - set({nodes[0]}) - set(solution)))]
  return solution

def distance(a,b,problem):
  return problem.get_weight(a,b)

def total_distance(solution, problem):
  total = 0
  for i in range(len(solution)-1):
    total += distance(solution[i] ,solution[i+1] ,  problem)
  return total + distance(solution[len(solution)-1] ,solution[0], problem)


solution = create_solution(nodes)
total_distance(solution,problem)

4841

In [7]:
def random_search(problem, n):
  best_solution = 0
  best_distance = 10e10

  for i in range(n):
    solution = create_solution(nodes)
    distance = total_distance(solution,problem)

    if distance < best_distance:
      best_solution = solution
      best_distance = distance

  return best_distance


random_search(problem, 10000)

3681

### Busqueda local

In [8]:
def generate_neighbour(solution):
  best_solution = []
  best_distance = 10e10

  for i in range(1, len(solution)-1):
    for j in range(i+i, len(solution)):

      neighbour = solution[:i] + [solution[j]] + solution[i+1:j] + [solution[i]] + solution[j+1:]
      neighbour_distance = total_distance(neighbour, problem)

      if neighbour_distance < best_distance:
        best_solution = neighbour
        best_distance = neighbour_distance

  return best_solution

solution = create_solution(nodes)
solution2 = generate_neighbour(solution) 
print(total_distance(solution, problem), total_distance(solution2,problem))

4718 4152


In [9]:
def local_search(problem):
  reference_solution = create_solution(nodes)
  best_distance = 10e10

  iteration = 0
  while(1):
    iteration += 1
    neighbour = generate_neighbour(reference_solution)
    neighbour_distance = total_distance(neighbour, problem)

    if neighbour_distance < best_distance:
      best_solution = neighbour
      best_distance = neighbour_distance
    else:
      print("En la iteración " + str(iteration) + " encontramos la solución " + str(best_distance))
      return best_distance

    reference_solution = neighbour

local_search(problem)

En la iteración 16 encontramos la solución 2519


2519

### Recocido simulado 

In [10]:
def generate_random_neighbour(solution):
  i,j = sorted(random.sample( range(1,len(solution)) , 2))
  return solution[:i] + [solution[j]] + solution[i+1:j] + [solution[i]] + solution[j+1:]

def probability(T,d):
  return random.random() < exp( -1*d / T)

def lower_temperature(T):
  return T*0.99

In [11]:
import copy

def simulated_annealing(problem, temperature=1000):
  reference_solution = create_solution(nodes)
  reference_distance = total_distance(reference_solution, problem)

  best_solution = []
  best_distance = 10e10

  iterations = 0
  while temperature > 0.0001:
    iterations += 1
    neighbour = generate_random_neighbour(reference_solution)
    neighbour_distance = total_distance(neighbour,problem)

    if neighbour_distance < best_distance:
      best_solution = neighbour
      best_distance = neighbour_distance

    if neighbour_distance < reference_distance or probability(temperature, abs(reference_distance - neighbour_distance)):
      reference_solution = copy.deepcopy(neighbour)
      reference_distance = neighbour_distance

    temperature = lower_temperature(temperature)
  
  return best_distance

simulated_annealing(problem)

2154

### Colonia de hormigas

In [12]:
def add_node(problem, H ,T ) :
  nodes = list(problem.get_nodes())
  return random.choice( list(set(range(1,len(nodes))) - set(H) ))

def increment_feromons(problem, T, H ) :
  for i in range(len(H)-1):
    T[H[i]][H[i+1]] += 1000/total_distance(H, problem)
  return T

def evaporate_feromones(T ):
  T = [[ max(T[i][j] - 0.3 , 1) for i in range(len(nodes)) ] for j in range(len(nodes))]
  return T

In [13]:
def ant_colony(problem, N) :
  nodes = list(problem.get_nodes())
  edges = list(problem.get_edges()) 
  
  T = [[ 1 for _ in range(len(nodes)) ] for _ in range(len(nodes))]
  
  #Se generan los agentes(hormigas) que serán estructuras de caminos desde 0
  ant = [[0] for _ in range(N)]
  
  #Recorre cada agente construyendo la solución
  for h in range(N) :
    #Para cada agente se construye un camino
    for i in range(len(nodes)-1) :
      
      #Elige el siguiente nodo
      new_node = add_node(problem, ant[h] ,T )
      ant[h].append(new_node)     
    
    #Incrementa feromonas en esa arista 
    T = increment_feromons(problem, T, ant[h] )
    #print("Feromonas(1)", T)
      
    #Evapora Feromonas  
    T = evaporate_feromones(T)

    #Seleccionamos el mejor agente
  best_solution = []
  best_distance = 10e100
  for h in range(N) :
    actual_distance = total_distance(ant[h], problem)
    if actual_distance < best_distance:
      best_solution = ant[h]
      best_distance = actual_distance
  
  
  print(best_solution)
  print(best_distance)
  
  
ant_colony(problem, 10000)

[0, 14, 12, 15, 7, 6, 41, 21, 9, 38, 26, 16, 31, 20, 18, 5, 27, 19, 13, 37, 35, 36, 23, 11, 25, 4, 22, 3, 1, 2, 28, 17, 32, 29, 10, 33, 34, 24, 8, 40, 39, 30]
3608
